In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csqa-logicalcombinations/test_all_hf.json
/kaggle/input/csqa-logicalcombinations/dev_all_hf.json
/kaggle/input/csqa-logicalcombinations/test_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/dev_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/train_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/train_all_hf.json
/kaggle/input/csqa-pairs-baselines/QA_Mixed.csv
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_NEITHER_results.csv
/kaggle/input/csqa-pairs-baselines/QA_NEITHER.csv
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_OR_results.csv
/kaggle/input/csqa-pairs-baselines/QA_AND.csv
/kaggle/input/csqa-pairs-baselines/__huggingface_repos__.json
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_MIXED_results.csv
/kaggle/input/csqa-pairs-baselines/csqa_zeroshot_AND_results.csv
/kaggle/input/csqa-pairs-baselines/QA_OR.csv
/kaggle/input

In [2]:
import json
import csv

# Read the JSONL file (one JSON object per line)
with open("/kaggle/input/csqa-logicalcombinations/dev_all_hf.json", "r") as f:
    data = [json.loads(line) for line in f]

# Write to CSV
with open('dev_dataset.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # Write header
    writer.writerow(['question', 'A', 'B', 'C', 'D', 'correct_label', 'correct_answer_text', 'qa_type'])
    
    # Write data rows
    for item in data:
        choices = item['choices']
        label = item['label']
        correct_answer = choices[label]
        
        # Map integer label to letter
        label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
        label_letter = label_map[label]
        
        correct_answer = choices[label]
        
        row = [
            item['question'],
            choices[0] if len(choices) > 0 else '',
            choices[1] if len(choices) > 1 else '',
            choices[2] if len(choices) > 2 else '',
            choices[3] if len(choices) > 3 else '',
            label_letter,  # Now shows A, B, C, or D
            correct_answer,
            item['qa_type']
        ]
        writer.writerow(row)

print("Conversion complete! Check dev_dataset.csv")




Conversion complete! Check dev_dataset.csv


In [3]:
qa_df = pd.read_csv('/kaggle/working/dev_dataset.csv')
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text,qa_type
0,Where can you see a mountain in your own home?,virtual art display AND themed room decor,decorative sculpture AND landscape design book,themed room decor AND landscape design book,mountain-themed restaurant AND outdoor camping...,A,virtual art display AND themed room decor,AND
1,Why should people try learning language for bu...,nuanced market understanding AND casual conver...,effective collaboration AND nuanced market und...,strategic relationship building AND casual con...,nuanced market understanding AND personal enjo...,B,effective collaboration AND nuanced market und...,AND
2,It was something that had weighed on him his w...,moment of joy AND final answer,path to freedom AND temporary relief,path to freedom AND source of growth,turning point AND temporary relief,C,path to freedom AND source of growth,AND
3,The restaurant had no business out in the coun...,town center AND quiet lakeside,busy marketplace AND remote cabin,remote cabin AND industrial park,town center AND near public transport,D,town center AND near public transport,AND
4,What can a country do when it has jealousy but...,manipulate alliances AND conduct covert operat...,conduct covert operations AND express competit...,manipulate alliances AND express competitive s...,express competitive spirit AND celebrate natio...,A,manipulate alliances AND conduct covert operat...,AND


In [4]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [6]:
from huggingface_hub import login
login(new_session=False)

In [7]:
# import pandas as pd
# import random
# import ast

# # Load your dataset
# df = pd.read_csv("/kaggle/working/logical_combinations_output.csv")  # replace with your file

# # Safe eval to ensure lists are properly loaded
# def safe_eval(val):
#     if isinstance(val, list):
#         return val
#     if pd.isna(val):
#         return []
#     if isinstance(val, str):
#         try:
#             return ast.literal_eval(val)
#         except:
#             return []
#     return []

# df['logical_combinations'] = df['logical_combinations'].apply(safe_eval)

# # Prepare storage for QA sets
# qa_sets = {
#     'AND': [],
#     'OR': [],
#     'NEITHER': [],
#     'Mixed': []
# }

# # Label rotation - separate rotator for each question type
# class LabelRotator:
#     def __init__(self, name):
#         self.name = name
#         self.labels = ['A', 'B', 'C', 'D']
#         self.current_index = 0
    
#     def get_next_label(self):
#         label = self.labels[self.current_index]
#         self.current_index = (self.current_index + 1) % len(self.labels)
#         # print(f"DEBUG {self.name}: Assigned label {label}, next will be {self.labels[self.current_index]}")
#         return label

# # Create separate rotators for each question type
# rotators = {
#     'AND': LabelRotator('AND'),
#     'OR': LabelRotator('OR'),
#     'NEITHER': LabelRotator('NEITHER'),
#     'Mixed': LabelRotator('Mixed')
# }

# def create_qa_item(question, correct_ans, incorrect_ans, qa_type):
#     """Helper function to create a QA item with proper label rotation"""
    
#     # Get correct label from the appropriate rotator
#     correct_label = rotators[qa_type].get_next_label()
    
#     # Ensure we have exactly 3 incorrect answers
#     if len(incorrect_ans) < 3:
#         print(f"Warning: Only {len(incorrect_ans)} incorrect answers available for {qa_type}")
#         return None
    
#     selected_incorrect = random.sample(incorrect_ans, 3)
    
#     # Create options array with 4 positions
#     options = [''] * 4
    
#     # Place correct answer at the designated position
#     correct_pos = rotators[qa_type].labels.index(correct_label)
#     options[correct_pos] = correct_ans
    
#     # Fill remaining positions with incorrect answers
#     incorrect_positions = [i for i in range(4) if i != correct_pos]
#     for i, pos in enumerate(incorrect_positions):
#         options[pos] = selected_incorrect[i]
    
#     return {
#         'question': question,
#         'A': options[0],
#         'B': options[1],
#         'C': options[2],
#         'D': options[3],
#         'correct_label': correct_label,
#         'correct_answer_text': correct_ans,
#         'qa_type': qa_type
#     }

# # Track counts for each type
# type_counts = {'AND': 0, 'OR': 0, 'NEITHER': 0, 'Mixed': 0}

# for idx, row in df.iterrows():
#     question = row['question']
#     combos = row['logical_combinations']
    
#     # Extract AND/OR/NEITHER correct and incorrect lists
#     and_correct = combos.get('AND_combinations', {}).get('correct', [])
#     and_incorrect = combos.get('AND_combinations', {}).get('incorrect', [])
    
#     or_correct = combos.get('OR_combinations', {}).get('correct', [])
#     or_incorrect = combos.get('OR_combinations', {}).get('incorrect', [])
    
#     neither_correct = combos.get('NEITHER_combinations', {}).get('correct', [])
#     neither_incorrect = combos.get('NEITHER_combinations', {}).get('incorrect', [])
    
#     # --- AND-only QA ---
#     if and_correct and len(and_incorrect) >= 3:
#         correct_ans = random.choice(and_correct)
#         qa_item = create_qa_item(question, correct_ans, and_incorrect, 'AND')
#         if qa_item:
#             qa_sets['AND'].append(qa_item)
#             type_counts['AND'] += 1
#             # print(f"AND Question {type_counts['AND']}: Label {qa_item['correct_label']}")
    
#     # --- OR-only QA ---
#     if or_correct and len(or_incorrect) >= 3:
#         correct_ans = random.choice(or_correct)
#         qa_item = create_qa_item(question, correct_ans, or_incorrect, 'OR')
#         if qa_item:
#             qa_sets['OR'].append(qa_item)
#             type_counts['OR'] += 1
#             # print(f"OR Question {type_counts['OR']}: Label {qa_item['correct_label']}")
    
#     # --- NEITHER-only QA ---
#     if neither_correct and len(neither_incorrect) >= 3:
#         correct_ans = random.choice(neither_correct)
#         qa_item = create_qa_item(question, correct_ans, neither_incorrect, 'NEITHER')
#         if qa_item:
#             qa_sets['NEITHER'].append(qa_item)
#             type_counts['NEITHER'] += 1
#             # print(f"NEITHER Question {type_counts['NEITHER']}: Label {qa_item['correct_label']}")
    
#     # --- Mixed QA ---
#     all_correct_types = [
#         ('AND', and_correct),
#         ('OR', or_correct),
#         ('NEITHER', neither_correct)
#     ]
#     all_incorrect_types = and_incorrect + or_incorrect + neither_incorrect
    
#     # Ensure at least one correct exists and enough incorrect answers
#     valid_categories = [cat for cat, lst in all_correct_types if lst]
#     if valid_categories and len(all_incorrect_types) >= 3:
#         chosen_category = random.choice(valid_categories)
#         chosen_correct_list = dict(all_correct_types)[chosen_category]
#         correct_ans = random.choice(chosen_correct_list)
#         qa_item = create_qa_item(question, correct_ans, all_incorrect_types, 'Mixed')
#         if qa_item:
#             qa_sets['Mixed'].append(qa_item)
#             type_counts['Mixed'] += 1
#             # print(f"Mixed Question {type_counts['Mixed']}: Label {qa_item['correct_label']}")

# print(f"\n=== GENERATION COMPLETE ===")
# for qtype, count in type_counts.items():
#     print(f"{qtype}: {count} questions generated")

# # --- Convert to DataFrames and save ---
# for key, qlist in qa_sets.items():
#     if qlist:  # Only save if we have questions
#         df_out = pd.DataFrame(qlist)
#         df_out.to_csv(f'QA_{key}.csv', index=False)
#         print(f"\n{key} QA saved: {len(df_out)} questions in QA_{key}.csv")
        
#         # Show label distribution for verification
#         label_counts = df_out['correct_label'].value_counts().sort_index()
#         print(f"  Label distribution: {dict(label_counts)}")
        
#         # Show label sequence for first 8 questions to verify rotation
#         print(f"  Label sequence (first 8): {list(df_out['correct_label'].head(8))}")
#     else:
#         print(f"{key}: No valid QA pairs generated")

# # Overall statistics
# total_questions = sum(len(qlist) for qlist in qa_sets.values())
# print(f"\nFinal total questions generated: {total_questions}")

In [8]:
# qa_df = pd.read_csv('/kaggle/input/csqa-pairs-baselines/QA_AND.csv')
qa_df.qa_type.unique()

array(['AND', 'OR', 'NEITHER', 'Mixed'], dtype=object)

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_qa_metrics(results_df):
    """
    
    Calculate comprehensive evaluation metrics for QA results
    
    Args:
        results_df: DataFrame with 'predicted_label' and 'correct_label' columns
    
    Returns:
        dict: Dictionary containing all evaluation metrics
    """
    
    # Remove None predictions for accurate calculation
    valid_predictions = results_df.dropna(subset=['predicted_label'])
    
    if len(valid_predictions) == 0:
        print("Warning: No valid predictions found!")
        return {}
    
    y_true = valid_predictions['correct_label'].tolist()
    y_pred = valid_predictions['predicted_label'].tolist()
    
    # Basic metrics
    total_questions = len(results_df)
    answered_questions = len(valid_predictions)
    unanswered_questions = total_questions - answered_questions
    
    # Accuracy (main metric for QA)
    accuracy = accuracy_score(y_true, y_pred)
    
    # Per-class metrics
    labels = ['A', 'B', 'C', 'D']
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )
    
    # Macro and micro averages
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    # Answer rate (percentage of questions answered)
    answer_rate = answered_questions / total_questions
    
    # Compile results
    metrics = {
        'total_questions': total_questions,
        'answered_questions': answered_questions,
        'unanswered_questions': unanswered_questions,
        'answer_rate': answer_rate,
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'micro_recall': micro_recall,
        'micro_f1': micro_f1,
    }
    
    # Per-class metrics
    for i, label in enumerate(labels):
        metrics[f'{label}_precision'] = precision[i]
        metrics[f'{label}_recall'] = recall[i]
        metrics[f'{label}_f1'] = f1[i]
        metrics[f'{label}_support'] = support[i]
    
    return metrics, cm, y_true, y_pred

def print_qa_metrics(metrics):
    """Pretty print the evaluation metrics"""
    print("=" * 50)
    print("QA EVALUATION METRICS")
    print("=" * 50)
    
    print(f"Total Questions: {metrics['total_questions']}")
    print(f"Answered Questions: {metrics['answered_questions']}")
    print(f"Answer Rate: {metrics['answer_rate']:.1%}")
    print(f"Accuracy: {metrics['accuracy']:.1%}")
    
    print("\n" + "-" * 30)
    print("OVERALL METRICS")
    print("-" * 30)
    print(f"Macro Precision: {metrics['macro_precision']:.3f}")
    print(f"Macro Recall: {metrics['macro_recall']:.3f}")
    print(f"Macro F1: {metrics['macro_f1']:.3f}")
    print(f"Micro Precision: {metrics['micro_precision']:.3f}")
    print(f"Micro Recall: {metrics['micro_recall']:.3f}")
    print(f"Micro F1: {metrics['micro_f1']:.3f}")
    
    print("\n" + "-" * 40)
    print("PER-CLASS METRICS")
    print("-" * 40)
    print(f"{'Class':<5} {'Precision':<9} {'Recall':<6} {'F1':<6} {'Support'}")
    print("-" * 40)
    
    for label in ['A', 'B', 'C', 'D']:
        precision = metrics[f'{label}_precision']
        recall = metrics[f'{label}_recall']
        f1 = metrics[f'{label}_f1']
        support = int(metrics[f'{label}_support'])
        print(f"{label:<5} {precision:<9.3f} {recall:<6.3f} {f1:<6.3f} {support}")

def plot_confusion_matrix(cm, labels=['A', 'B', 'C', 'D'], save_path=None):
    """Plot confusion matrix"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def analyze_answer_distribution(results_df):
    """Analyze the distribution of correct and predicted answers"""
    print("\n" + "=" * 50)
    print("ANSWER DISTRIBUTION ANALYSIS")
    print("=" * 50)
    
    # True label distribution
    true_dist = results_df['correct_label'].value_counts().sort_index()
    print("\nTrue Label Distribution:")
    for label in ['A', 'B', 'C', 'D']:
        count = true_dist.get(label, 0)
        pct = count / len(results_df) * 100
        print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Predicted label distribution (excluding None)
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        pred_dist = valid_preds['predicted_label'].value_counts().sort_index()
        print("\nPredicted Label Distribution:")
        for label in ['A', 'B', 'C', 'D']:
            count = pred_dist.get(label, 0)
            pct = count / len(valid_preds) * 100
            print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Check for bias towards certain options
    print("\nLabel Rotation Check:")
    if len(set(true_dist.values)) == 1:
        print("✓ Perfect label rotation detected")
    else:
        print("⚠ Uneven label distribution detected")
        
def comprehensive_qa_evaluation(results_df, save_plots=True):
    """Run complete evaluation pipeline"""
    
    # Calculate metrics
    metrics, cm, y_true, y_pred = calculate_qa_metrics(results_df)
    
    if not metrics:
        return
    
    # Print metrics
    print_qa_metrics(metrics)
    
    # Analyze distributions
    analyze_answer_distribution(results_df)
    
    # Plot confusion matrix
    if save_plots:
        plot_confusion_matrix(cm, save_path='confusion_matrix.png')
    else:
        plot_confusion_matrix(cm)
    
    # Print sklearn classification report for additional insights
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        print("\n" + "=" * 50)
        print("DETAILED CLASSIFICATION REPORT")
        print("=" * 50)
        print(classification_report(
            valid_preds['correct_label'], 
            valid_preds['predicted_label'],
            labels=['A', 'B', 'C', 'D']
        ))
    
    return metrics

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")

2025-11-20 20:47:18.386593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763671638.571498      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763671638.623725      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [11]:
def zero_shot_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:
            # More explicit prompt for single character output
#             prompt = f"""Answer the following commonsense question.
            
# Question: {str(row['question'])}
# A. {str(row['A'])}
# B. {str(row['B'])}
# C. {str(row['C'])}
# D. {str(row['D'])}
# Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
# Answer:"""

            prompt = f"""Answer the following commonsense question by selecting the correct option.
            Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.

Question: {str(row['question'])}

(A) {str(row['A'])}
(B) {str(row['B'])}
(C) {str(row['C'])}
(D) {str(row['D'])}

Answer:"""

            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=2,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            
            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [12]:
import re

def two_shot_and_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Think step by step, then give ONLY the single, capital letter (A, B, C, or D) as your final answer.
                
                Examples:
                
                Question: Sammy wanted to go to where the people were. Where might he go?
                A. social venues AND quiet retreats
                B. local events AND social venues
                C. sports arenas AND quiet retreats
                D. sports arenas AND train platforms
                
                Let's think step by step.
                Sammy wants to go where people are. "Quiet retreats" have few people, so A and C are wrong. 
                "Train platforms" have people but aren't for socializing, so D is less fitting. 
                "Local events" and "social venues" both have people. Therefore, the correct answer is B.
                
                Answer: B
                
                Question: The fox walked from the city into the forest, what was it looking for?
                A. suitable prey AND shelter from disturbances
                B. urban garden AND farm fields
                C. natural habitat AND farm fields
                D. suitable prey AND neighborhood pets
                
                Let's think step by step.
                The fox leaves the city for the forest, so options involving "urban" or "neighborhood" don't fit. 
                "Farm fields" are not in forests. "Suitable prey" and "shelter from disturbances" both fit forest life. 
                Therefore, the correct answer is A.
                
                Answer: A
                
                Now answer the following question:
                
                Question: {str(row['question'])}
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}
                Let's think step by step.
                """


            import time
            start = time.time()
            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=128,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            generated_text = generated_text.strip()
            print(generated_text)
            answer_match = re.search(r'Answer\s*:\s*([A-D])', generated_text, re.IGNORECASE)
            if answer_match:
                answer = answer_match.group(1).upper()
                    
                # Validate that the first character is a valid choice
                if answer in valid_choices:
                    predicted_label = answer
            else:

                final_answer_patterns = [
                r'(?:therefore|thus|so|hence)\s*,?\s*(?:the answer is\s*)?([A-D])\b',
                r'([A-D])\s+is\s+(?:the\s+)?correct',
                r'(?:option|answer|choice)\s+([A-D])\s+is',
                r'the\s+answer\s+is\s+([A-D])',
                ]
            
                for pattern in final_answer_patterns:
                    match = re.search(pattern, generated_text, re.IGNORECASE)
                    if match:
                        answer = match.group(1).upper()
                        if answer in valid_choices:
                            predicted_label = answer
                if not predicted_label:
                    
                    # Strategy 3: Look ONLY in last 30 characters for standalone letter
                    # This avoids picking letters from the reasoning itself
                    last_segment = generated_text[-30:].strip()
                    
                    # Must be standalone (word boundary on both sides)
                    standalone_match = re.search(r'\b([A-D])\b', last_segment, re.IGNORECASE)
                    if standalone_match:
                        answer =  standalone_match.group(1).upper()
                        if answer in valid_choices:
                            predicted_label = answer
            

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
        end = time.time()
        print(end-start)
    return results

In [13]:
qa_df_AND = qa_df[qa_df.qa_type == 'AND']
results = zero_shot_qa(model, tokenizer, qa_df_AND)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_AND_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)

for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for op

({'total_questions': 1500, 'answered_questions': 1498, 'unanswered_questions': 2, 'answer_rate': 0.9986666666666667, 'accuracy': 0.6942590120160214, 'macro_precision': 0.7138373155347105, 'macro_recall': 0.6943806970509383, 'macro_f1': 0.6955793797319922, 'micro_precision': 0.6942590120160214, 'micro_recall': 0.6942590120160214, 'micro_f1': 0.6942590120160214, 'A_precision': 0.8129496402877698, 'A_recall': 0.6026666666666667, 'A_f1': 0.6921898928024502, 'A_support': 375, 'B_precision': 0.7888198757763976, 'B_recall': 0.6773333333333333, 'B_f1': 0.7288378766140603, 'B_support': 375, 'C_precision': 0.6592592592592592, 'C_recall': 0.712, 'C_f1': 0.6846153846153845, 'C_support': 375, 'D_precision': 0.5943204868154158, 'D_recall': 0.7855227882037533, 'D_f1': 0.6766743648960739, 'D_support': 373}, array([[226,  21,  53,  75],
       [ 19, 254,  41,  61],
       [ 15,  29, 267,  64],
       [ 18,  18,  44, 293]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D'

In [14]:
def two_shot_or_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Think step by step, then give ONLY the single, capital letter (A, B, C, or D) as your final answer.
                Note: Each option contains two parts joined by OR and the answer is correct if even one part makes sense.
                
                Examples:
                
                Question: Sammy wanted to go to where the people were. Where might he go?
                A. social venues OR quiet retreats
                B. local events OR social venues
                C. sports arenas OR quiet retreats
                D. sports arenas OR train platforms
                
                Let's think step by step.
                Sammy wants to go where people are. "Quiet retreats" have few people, so A and C are wrong. 
                "Train platforms" have people but aren't for socializing, so D is less fitting. 
                "Local events" and "social venues" both have people. Therefore, the correct answer is B.
                
                Answer: B
                
                Question: The fox walked from the city into the forest, what was it looking for?
                A. suitable prey OR shelter from disturbances
                B. urban garden OR farm fields
                C. natural habitat OR farm fields
                D. suitable prey OR neighborhood pets
                
                Let's think step by step.
                The fox leaves the city for the forest, so options involving "urban" or "neighborhood" don't fit. 
                "Farm fields" are not in forests. "Suitable prey" or "shelter from disturbances" both fit forest life, so A is correct.
                
                Answer: A
                
                Now answer the following question:
                
                Question: {str(row['question'])}
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}
                Let's think step by step.
                """


            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=256,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            generated_text = generated_text.strip()
            
            answer_match = re.search(r'Answer\s*:\s*([A-D])', generated_text, re.IGNORECASE)
            if answer_match:
                answer = answer_match.group(1).upper()
                    
                # Validate that the first character is a valid choice
                if answer in valid_choices:
                    predicted_label = answer
            else:

                final_answer_patterns = [
                r'(?:therefore|thus|so|hence)\s*,?\s*(?:the answer is\s*)?([A-D])\b',
                r'([A-D])\s+is\s+(?:the\s+)?correct',
                r'(?:option|answer|choice)\s+([A-D])\s+is',
                r'the\s+answer\s+is\s+([A-D])',
                ]
            
                for pattern in final_answer_patterns:
                    match = re.search(pattern, generated_text, re.IGNORECASE)
                    if match:
                        answer = match.group(1).upper()
                        if answer in valid_choices:
                            predicted_label = answer
                if not predicted_label:
                    
                    # Strategy 3: Look ONLY in last 30 characters for standalone letter
                    # This avoids picking letters from the reasoning itself
                    last_segment = generated_text[-30:].strip()
                    
                    # Must be standalone (word boundary on both sides)
                    standalone_match = re.search(r'\b([A-D])\b', last_segment, re.IGNORECASE)
                    if standalone_match:
                        answer =  standalone_match.group(1).upper()
                        if answer in valid_choices:
                            predicted_label = answer
            

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [15]:
qa_df_OR = qa_df[qa_df.qa_type == 'OR']
results = zero_shot_qa(model, tokenizer, qa_df_OR)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_OR_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 1500, 'answered_questions': 1498, 'unanswered_questions': 2, 'answer_rate': 0.9986666666666667, 'accuracy': 0.6288384512683578, 'macro_precision': 0.6392471953909582, 'macro_recall': 0.6288627450980393, 'macro_f1': 0.6298247966745429, 'micro_precision': 0.6288384512683578, 'micro_recall': 0.6288384512683578, 'micro_f1': 0.6288384512683578, 'A_precision': 0.7237762237762237, 'A_recall': 0.552, 'A_f1': 0.6263237518910741, 'A_support': 375, 'B_precision': 0.6952908587257618, 'B_recall': 0.6693333333333333, 'B_f1': 0.6820652173913043, 'B_support': 375, 'C_precision': 0.5813397129186603, 'C_recall': 0.6497326203208557, 'C_f1': 0.6136363636363636, 'C_support': 374, 'D_precision': 0.5565819861431871, 'D_recall': 0.6443850267379679, 'D_f1': 0.59727385377943, 'D_support': 374}, array([[207,  44,  68,  56],
       [ 23, 251,  36,  65],
       [ 26,  34, 243,  71],
       [ 30,  32,  71, 241]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 

In [16]:
def two_shot_nnor_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Think step by step, then give ONLY the single, capital letter (A, B, C, or D) as your final answer.
                Note: Each option uses NEITHER–NOR and the correct answer is the one where both parts do NOT make sense together in the situation.
                
                Examples:
                
                Question: Sammy wanted to go to where the people were. Where might he go? 
                A. NEITHER local events NOR train platforms 
                B. NEITHER sports arenas NOR train platforms 
                C. NEITHER social venues NOR sports arenas 
                D. NEITHER social venues NOR quiet retreats
                
                Let's think step by step.
                Sammy wants to be where people are. “Quiet retreats” have few people, and “social venues” or “sports arenas” have many, so C and D make no sense.  
                Train platforms have people, but they’re not social, and “sports arenas” fit the opposite intent.  
                “NEITHER sports arenas NOR train platforms” both go against the goal of finding people.  
                Therefore, the correct answer is B.
                
                Answer: B
                
                Question: The forgotten leftovers had gotten quite old, he found it covered in mold in the back of his what?
                A. NEITHER cold storage NOR dining area
                B. NEITHER sealed container NOR living room
                C. NEITHER food cabinet NOR utility drawer
                D. NEITHER living room NOR utility drawer
                
                Let's think step by step.
                Moldy leftovers are usually found in a refrigerator or cold storage, not a living room or utility drawer.  
                Option D fits because neither of those places is for storing food.  
                Therefore, the correct answer is D.
                
                Answer: D
                
                Now answer the following question:
                
                Question: {str(row['question'])}
                A. {str(row['A'])}
                B. {str(row['B'])}
                C. {str(row['C'])}
                D. {str(row['D'])}
                Let's think step by step.
                """



            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=256,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            generated_text = generated_text.strip()
            
            answer_match = re.search(r'Answer\s*:\s*([A-D])', generated_text, re.IGNORECASE)
            if answer_match:
                answer = answer_match.group(1).upper()
                    
                # Validate that the first character is a valid choice
                if answer in valid_choices:
                    predicted_label = answer
            else:

                final_answer_patterns = [
                r'(?:therefore|thus|so|hence)\s*,?\s*(?:the answer is\s*)?([A-D])\b',
                r'([A-D])\s+is\s+(?:the\s+)?correct',
                r'(?:option|answer|choice)\s+([A-D])\s+is',
                r'the\s+answer\s+is\s+([A-D])',
                ]
            
                for pattern in final_answer_patterns:
                    match = re.search(pattern, generated_text, re.IGNORECASE)
                    if match:
                        answer = match.group(1).upper()
                        if answer in valid_choices:
                            predicted_label = answer
                if not predicted_label:
                    
                    # Strategy 3: Look ONLY in last 30 characters for standalone letter
                    # This avoids picking letters from the reasoning itself
                    last_segment = generated_text[-30:].strip()
                    
                    # Must be standalone (word boundary on both sides)
                    standalone_match = re.search(r'\b([A-D])\b', last_segment, re.IGNORECASE)
                    if standalone_match:
                        answer =  standalone_match.group(1).upper()
                        if answer in valid_choices:
                            predicted_label = answer
            

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [17]:
qa_df_NEITHER = qa_df[qa_df.qa_type == 'NEITHER']
results = zero_shot_qa(model, tokenizer, qa_df_NEITHER)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_NEITHER_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 1500, 'answered_questions': 1468, 'unanswered_questions': 32, 'answer_rate': 0.9786666666666667, 'accuracy': 0.11376021798365123, 'macro_precision': 0.11405499037505618, 'macro_recall': 0.11376648111508689, 'macro_f1': 0.11085298758657125, 'micro_precision': 0.11376021798365123, 'micro_recall': 0.11376021798365123, 'micro_f1': 0.11376021798365124, 'A_precision': 0.17088607594936708, 'A_recall': 0.14917127071823205, 'A_f1': 0.15929203539823011, 'A_support': 362, 'B_precision': 0.08717948717948718, 'B_recall': 0.04632152588555858, 'B_f1': 0.060498220640569395, 'B_support': 367, 'C_precision': 0.09068627450980392, 'C_recall': 0.10054347826086957, 'C_f1': 0.09536082474226804, 'C_support': 368, 'D_precision': 0.10746812386156648, 'D_recall': 0.15902964959568733, 'D_f1': 0.1282608695652174, 'D_support': 371}, array([[ 54,  53, 114, 141],
       [ 75,  17, 117, 158],
       [ 88,  52,  37, 191],
       [ 99,  73, 140,  59]]), ['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 

In [18]:
def three_shot_mixed_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""
                Answer the following commonsense question by selecting the correct option.
                Instructions: Please respond with ONLY the single, capital letter (A, B, C, or D) that is the correct answer. Do not include any other text, punctuation, or explanation.
                
                Examples:
                   Question: The fox walked from the city into the forest, what was it looking for? 
        
                    A. natural habitat AND shelter from disturbances
                    B. farm fields OR neighborhood pets
                    C. city park AND neighborhood pets
                    D. farm fields AND neighborhood pets
                        
                Answer: A

                Question: Google Maps and other highway and street GPS services have replaced what?
                    A. historical navigation logs AND tourist information centers
                    B. NEITHER printed road maps NOR route planning software
                    C. manual navigation techniques AND tourist information centers
                    D. manual navigation techniques AND route planning software
                    
                    Answer: D

                Question: Where is a business restaurant likely to be located?
                    A. corporate office clusters AND at university campuses
                    B. business park entrances AND near conference venues
                    C. NEITHER near conference venues NOR at university campuses
                    D. NEITHER near conference venues NOR near tourist sites
                    
                    Answer: B
                
                Now answer the following question:
                
                Question: {str(row['question'])}
                (A) {str(row['A'])}
                (B) {str(row['B'])}
                (C) {str(row['C'])}
                (D) {str(row['D'])}
                Answer:"""


            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=2,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            
            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}"""
        })
            
    return results

In [19]:
qa_df_MIXED = qa_df[qa_df.qa_type == 'Mixed']
results = zero_shot_qa(model, tokenizer, qa_df_MIXED)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_MIXED_results.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)


for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 1500, 'answered_questions': 1470, 'unanswered_questions': 30, 'answer_rate': 0.98, 'accuracy': 0.4605442176870748, 'macro_precision': 0.4682286264338434, 'macro_recall': 0.4599067168432638, 'macro_f1': 0.4572363274238349, 'micro_precision': 0.4605442176870748, 'micro_recall': 0.4605442176870748, 'micro_f1': 0.4605442176870748, 'A_precision': 0.5076923076923077, 'A_recall': 0.36164383561643837, 'A_f1': 0.4224, 'A_support': 365, 'B_precision': 0.48494983277591974, 'B_recall': 0.3983516483516483, 'B_f1': 0.4374057315233785, 'B_support': 364, 'C_precision': 0.46064814814814814, 'C_recall': 0.5363881401617251, 'C_f1': 0.49564134495641343, 'C_support': 371, 'D_precision': 0.4196242171189979, 'D_recall': 0.5432432432432432, 'D_f1': 0.47349823321554774, 'D_support': 370}, array([[132,  55,  72, 106],
       [ 44, 145,  90,  85],
       [ 35,  50, 199,  87],
       [ 49,  49,  71, 201]]), ['A', 'B', 'C', 'D', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 